In [1]:
!git clone https://github.com/moienr/YOLO_Road_Damage.git

Cloning into 'YOLO_Road_Damage'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 0), reused 11 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 10.74 KiB | 10.74 MiB/s, done.


In [2]:
import sys

In [3]:
# Add the cloned repository to the Python path
sys.path.append('/content/YOLO_Road_Damage')

In [4]:
from dataset import RDD_CLASSES
print(RDD_CLASSES)

['D00', 'D10', 'D20', 'D40', 'Repair', 'Block crack']


In [5]:
!wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_China_Drone.zip

--2023-12-09 14:12:22--  https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_China_Drone.zip
Resolving bigdatacup.s3.ap-northeast-1.amazonaws.com (bigdatacup.s3.ap-northeast-1.amazonaws.com)... 52.219.1.123, 52.219.8.162, 52.219.152.150, ...
Connecting to bigdatacup.s3.ap-northeast-1.amazonaws.com (bigdatacup.s3.ap-northeast-1.amazonaws.com)|52.219.1.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160193541 (153M) [application/zip]
Saving to: ‘RDD2022_China_Drone.zip’

RDD2022_China_Drone 100%[===================>] 152.77M  29.1MB/s    in 5.1s    

2023-12-09 14:12:27 (29.9 MB/s) - ‘RDD2022_China_Drone.zip’ saved [160193541/160193541]



In [6]:
%%capture
!unzip /content/RDD2022_China_Drone.zip

In [7]:
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from tqdm import tqdm
from torch.utils.data import DataLoader
from model import Yolov1
from dataset import VOCDataset
from utils import (
    non_max_suppression,
    mean_average_precision,
    intersection_over_union,
    cellboxes_to_boxes,
    get_bboxes,
    plot_image,
    save_checkpoint,
    load_checkpoint,
)
from loss import YoloLoss

seed = 123
torch.manual_seed(seed)

In [17]:
# Hyperparameters etc.
LEARNING_RATE = 2e-4
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
BATCH_SIZE = 16 # 64 in original paper but I don't have that much vram, grad accum?
WEIGHT_DECAY = 0
EPOCHS = 100
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "overfit.pth.tar"
IMG_DIR = "/content/China_Drone/train/images"
LABEL_DIR = "/content/China_Drone/train/annotations/xmls"


In [18]:


class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, bboxes):
        for t in self.transforms:
            img, bboxes = t(img), bboxes

        return img, bboxes


transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor(),])


def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    mean_loss = []

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update progress bar
        loop.set_postfix(loss=loss.item())

    print(f"Mean loss was {sum(mean_loss)/len(mean_loss)}")


def main():
    model = Yolov1(split_size=7, num_boxes=2, num_classes=6).to(DEVICE)
    optimizer = optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )
    loss_fn = YoloLoss()

    if LOAD_MODEL:
        load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

    train_dataset = VOCDataset(
        transform=transform,
        img_dir=IMG_DIR,
        label_dir=LABEL_DIR,
    )

    test_dataset = VOCDataset( transform=transform, img_dir=IMG_DIR, label_dir=LABEL_DIR,)

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

    for epoch in range(EPOCHS):
        # for x, y in train_loader:
        #    x = x.to(DEVICE)
        #    for idx in range(8):
        #        bboxes = cellboxes_to_boxes(model(x))
        #        bboxes = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
        #        plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes)

        #    import sys
        #    sys.exit()

        pred_boxes, target_boxes = get_bboxes(
            train_loader, model, iou_threshold=0.5, threshold=0.4
        )

        mean_avg_prec = mean_average_precision(
            pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
        )
        print(f"Train mAP: {mean_avg_prec}")

        # if mean_avg_prec > 0.9:
        if epoch == EPOCHS - 1:
           checkpoint = {
               "state_dict": model.state_dict(),
               "optimizer": optimizer.state_dict(),
           }
           save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE)
           import time
           time.sleep(10)

        train_fn(train_loader, model, optimizer, loss_fn)


In [21]:
model = Yolov1(split_size=7, num_boxes=2, num_classes=6).to(DEVICE)
optimizer = optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )
loss_fn = YoloLoss()

In [22]:
load_checkpoint(torch.load("/content/overfit.pth.tar"), model, optimizer)

=> Loading checkpoint


In [24]:
import utils

In [36]:
train_dataset = VOCDataset(
    transform=transform,
    img_dir=IMG_DIR,
    label_dir=LABEL_DIR,
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=False,
    drop_last=True,
)

In [37]:
all_pred_boxes, all_true_boxes = utils.get_bboxes(train_loader,model,iou_threshold=0.5, threshold=0.4)

In [38]:
len(all_pred_boxes), len(all_true_boxes)

(24139, 3811)

In [39]:
ix = 2
all_pred_boxes[ix],all_true_boxes[ix]

([1,
  0.0,
  0.558247983455658,
  0.5696244835853577,
  0.14660102128982544,
  0.012811226770281792,
  -0.00439764279872179],
 [2,
  1.0,
  1.0,
  0.4111328423023224,
  0.3662109673023224,
  0.7324219346046448,
  0.4316406548023224])